## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-07-16-53-48 +0000,bbc,When is the Budget and what might be in it?,https://www.bbc.com/news/articles/cj4w44w42j5o...
1,2025-10-07-16-52-49 +0000,nyt,White House May Try to Deny Back Pay to Furlou...,https://www.nytimes.com/2025/10/07/us/politics...
2,2025-10-07-16-47-01 +0000,nypost,University student plans to restore Christian ...,https://nypost.com/2025/10/07/media/university...
3,2025-10-07-16-46-35 +0000,nyt,Deadly Clashes Erupt Between Syrian Government...,https://www.nytimes.com/2025/10/07/world/middl...
4,2025-10-07-16-46-25 +0000,bbc,Payouts of £700 per driver after car finance s...,https://www.bbc.com/news/articles/cqlzwqv7xz1o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2325/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
66,trump,58
291,new,22
315,gaza,17
43,israel,15
70,shutdown,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
217,2025-10-06-22-57-48 +0000,nypost,Senate Democrats reject latest effort to end g...,https://nypost.com/2025/10/06/us-news/senate-d...,129
322,2025-10-06-17-52-24 +0000,nypost,Retired IDF general who saved his family on Oc...,https://nypost.com/2025/10/06/world-news/retir...,119
234,2025-10-06-22-14-52 +0000,latimes,Trump slams judge he picked as court tests lim...,https://www.latimes.com/california/story/2025-...,117
284,2025-10-06-20-08-48 +0000,bbc,New US Supreme Court term will reshape Trump's...,https://www.bbc.com/news/articles/cly959ppezeo...,114
285,2025-10-06-20-04-32 +0000,nypost,Trump appealing ruling against Portland Nation...,https://nypost.com/2025/10/06/us-news/trump-ap...,114


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
217,129,2025-10-06-22-57-48 +0000,nypost,Senate Democrats reject latest effort to end g...,https://nypost.com/2025/10/06/us-news/senate-d...
224,74,2025-10-06-22-37-00 +0000,wsj,The Reserve Bank of New Zealand has establishe...,https://www.wsj.com/articles/new-zealand-centr...
167,61,2025-10-07-08-32-25 +0000,bbc,"Jeremy Bowen: Two years on, will Israel and Ha...",https://www.bbc.com/news/articles/cvgqyj268ljo...
285,56,2025-10-06-20-04-32 +0000,nypost,Trump appealing ruling against Portland Nation...,https://nypost.com/2025/10/06/us-news/trump-ap...
295,52,2025-10-06-19-34-39 +0000,cbc,Bari Weiss named editor-in-chief of CBS News u...,https://www.cbc.ca/news/entertainment/bari-wei...
322,48,2025-10-06-17-52-24 +0000,nypost,Retired IDF general who saved his family on Oc...,https://nypost.com/2025/10/06/world-news/retir...
299,42,2025-10-06-19-11-11 +0000,bbc,US Supreme Court rejects Ghislaine Maxwell app...,https://www.bbc.com/news/articles/cn83e6q7lq4o...
62,38,2025-10-07-14-35-20 +0000,nypost,Virginia Fraternal Order of Police demands ‘un...,https://nypost.com/2025/10/07/us-news/virginia...
33,36,2025-10-07-15-50-55 +0000,nypost,NY woman refuses to pay $440 fine to Hertz aft...,https://nypost.com/2025/10/07/business/ny-woma...
75,32,2025-10-07-13-55-59 +0000,nypost,Disturbed Staten Island teen who allegedly dec...,https://nypost.com/2025/10/07/us-news/disturbe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
